In [ ]:
# Install vLLM and its dependencies
!pip install vllm transformers accelerate einops sentencepiece xformers
!pip install autoawq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 123.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoawq 0.2.8 requires transformers<=4.47.1,>=4.45.0, but you have transformers 4.51.3 which is incompatible.
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follo

In [ ]:
from vllm import LLM, SamplingParams
import torch
import numpy as np
import math
import awq

In [ ]:
# Using a pre-quantized AWQ model
llm = LLM(
    model='TheBloke/deepseek-coder-1.3b-instruct-AWQ',
    gpu_memory_utilization=0.8,
    max_model_len=2048,  # Reduce context length to save memory
    quantization='awq'   # Now this will work with the pre-quantized model
)

INFO 04-24 00:12:38 [config.py:689] This model supports multiple tasks: {'classify', 'score', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
WARNING 04-24 00:12:38 [config.py:768] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-24 00:12:38 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='TheBloke/deepseek-coder-1.3b-instruct-AWQ', speculative_config=None, tokenizer='TheBloke/deepseek-coder-1.3b-instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observabil

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-24 00:12:44 [loader.py:458] Loading weights took 3.66 seconds
INFO 04-24 00:12:44 [model_runner.py:1146] Model loading took 0.8345 GiB and 4.206283 seconds
INFO 04-24 00:12:46 [worker.py:267] Memory profiling takes 0.81 seconds
INFO 04-24 00:12:46 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.80) = 11.79GiB
INFO 04-24 00:12:46 [worker.py:267] model weights take 0.83GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.30GiB; the rest of the memory reserved for KV Cache is 10.66GiB.
INFO 04-24 00:12:46 [executor_base.py:112] # cuda blocks: 3637, # CPU blocks: 1365
INFO 04-24 00:12:46 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 28.41x


OutOfMemoryError: CUDA out of memory. Tried to allocate 456.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 208.12 MiB is free. Process 32483 has 14.52 GiB memory in use. Of the allocated memory 14.05 GiB is allocated by PyTorch, with 92.00 MiB allocated in private pools (e.g., CUDA Graphs), and 163.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def calculate_perplexity(model_output):
    """Calculate perplexity from token logprobs in vLLM output"""
    if not model_output or len(model_output) == 0:
        return None, []

    output_obj = model_output[0]
    token_logprobs = []

    # Extract token logprobs from the nested structure
    if hasattr(output_obj, 'outputs') and output_obj.outputs:
        for output in output_obj.outputs:
            if hasattr(output, 'logprobs'):
                for i, logprob_dict in enumerate(output.logprobs):
                    # Extract the top-ranked token's logprob
                    token_id = output.token_ids[i]
                    if token_id in logprob_dict:
                        token_logprobs.append(logprob_dict[token_id].logprob)

    if not token_logprobs:
        return None, []

    # Calculate perplexity
    avg_logprob = sum(token_logprobs) / len(token_logprobs)
    perplexity = math.exp(-avg_logprob)

    return perplexity, token_logprobs

def get_token_probabilities(prompt, max_tokens=15):
    """Get token probabilities for a given prompt"""
    sampling_params = SamplingParams(
        max_tokens=max_tokens,
        temperature=0,
        logprobs=5
    )

    outputs = llm.generate([prompt], sampling_params)

    # Calculate perplexity
    perplexity, token_logprobs = calculate_perplexity(outputs)

    # Extract completion text
    completion = ""
    if outputs and len(outputs) > 0 and outputs[0].outputs:
        completion = outputs[0].outputs[0].text

    # Extract token texts for better interpretation
    tokens = []
    if outputs and len(outputs) > 0 and outputs[0].outputs:
        for i, logprob_dict in enumerate(outputs[0].outputs[0].logprobs):
            token_id = outputs[0].outputs[0].token_ids[i]
            if token_id in logprob_dict:
                top_token = logprob_dict[token_id]
                tokens.append(top_token.decoded_token)

    return {
        "completion": completion,
        "perplexity": perplexity,
        "token_logprobs": token_logprobs,
        "tokens": tokens,
        "avg_logprob": sum(token_logprobs) / len(token_logprobs) if token_logprobs else None
    }

# Compare perplexity across different prompts
prompts = [
  "The capital of France is"  # Non-coding example
]

results = []
for prompt in prompts:
    result = get_token_probabilities(prompt, max_tokens=15)
    results.append(result)

    # Print in a readable format
    print(f"\n== Prompt: {prompt} ==")
    print(f"Completion: {result['completion']}")
    print(f"Perplexity: {result['perplexity']:.4f}" if result['perplexity'] else "Perplexity: None")
    print(f"Average logprob: {result['avg_logprob']:.4f}" if result['avg_logprob'] else "Average logprob: None")

    # Print token-by-token breakdown
    if result['tokens'] and result['token_logprobs']:
        print("\nToken breakdown:")
        for i, (token, logprob) in enumerate(zip(result['tokens'], result['token_logprobs'])):
            print(f"  Token {i+1}: '{token}' - logprob: {logprob:.4f}, probability: {math.exp(logprob):.4f}")

# Compare perplexities
if all(r['perplexity'] for r in results):
    print("\n== Perplexity Comparison ==")
    for prompt, result in zip(prompts, results):
        print(f"{prompt}: {result['perplexity']:.4f}")

    # Find the prompt with lowest perplexity (most predictable output)
    most_predictable = min(results, key=lambda x: x['perplexity'] if x['perplexity'] else float('inf'))
    print(f"\nMost predictable prompt: {prompts[results.index(most_predictable)]}")
    print(f"Perplexity: {most_predictable['perplexity']:.4f}")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


== Prompt: def calculate_fibonacci(n): ==
Completion: 
    if n <= 0:
        return "输入无效，
Perplexity: 1.5500
Average logprob: -0.4383

Token breakdown:
  Token 1: '
' - logprob: -0.0388, probability: 0.9619
  Token 2: '   ' - logprob: -1.7615, probability: 0.1718
  Token 3: ' if' - logprob: -0.4684, probability: 0.6260
  Token 4: ' n' - logprob: -0.0164, probability: 0.9838
  Token 5: ' <= ' - logprob: -0.4565, probability: 0.6335
  Token 6: '0' - logprob: -0.2009, probability: 0.8180
  Token 7: ':' - logprob: -0.0016, probability: 0.9984
  Token 8: '
' - logprob: -0.0126, probability: 0.9875
  Token 9: '       ' - logprob: -0.0090, probability: 0.9911
  Token 10: ' return' - logprob: -0.1714, probability: 0.8425
  Token 11: ' "' - logprob: -0.3778, probability: 0.6854
  Token 12: '输入' - logprob: -1.0887, probability: 0.3367
  Token 13: '无' - logprob: -0.8833, probability: 0.4134
  Token 14: '效' - logprob: -0.0000, probability: 1.0000
  Token 15: '，' - logprob: -1.0870, probability:

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


== Prompt: class BinarySearchTree: ==
Completion: 


    def __init__(self, value):
        self
Perplexity: 1.3092
Average logprob: -0.2694

Token breakdown:
  Token 1: '
' - logprob: -0.0298, probability: 0.9707
  Token 2: '
' - logprob: -1.1340, probability: 0.3217
  Token 3: '
' - logprob: -1.2243, probability: 0.2940
  Token 4: '   ' - logprob: -0.2147, probability: 0.8068
  Token 5: ' def' - logprob: -0.4034, probability: 0.6680
  Token 6: ' __' - logprob: -0.0018, probability: 0.9982
  Token 7: 'init' - logprob: -0.0001, probability: 0.9999
  Token 8: '__(' - logprob: -0.0003, probability: 0.9997
  Token 9: 'self' - logprob: -0.0003, probability: 0.9997
  Token 10: ',' - logprob: -0.2387, probability: 0.7876
  Token 11: ' value' - logprob: -0.6180, probability: 0.5390
  Token 12: '):' - logprob: -0.1645, probability: 0.8484
  Token 13: '
' - logprob: -0.0022, probability: 0.9978
  Token 14: '       ' - logprob: -0.0056, probability: 0.9944
  Token 15: ' self' - logprob: -0.0034

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


== Prompt: import torch
def train_model( ==
Completion: model, dataloader, loss_fn, optimizer):

Perplexity: 1.2155
Average logprob: -0.1952

Token breakdown:
  Token 1: 'model' - logprob: -0.2199, probability: 0.8026
  Token 2: ',' - logprob: -0.0246, probability: 0.9757
  Token 3: ' dat' - logprob: -1.2499, probability: 0.2865
  Token 4: 'al' - logprob: -0.0018, probability: 0.9982
  Token 5: 'oad' - logprob: -0.0000, probability: 1.0000
  Token 6: 'er' - logprob: -0.1164, probability: 0.8901
  Token 7: ',' - logprob: -0.0093, probability: 0.9908
  Token 8: ' loss' - logprob: -0.6044, probability: 0.5464
  Token 9: '_' - logprob: -0.0006, probability: 0.9994
  Token 10: 'fn' - logprob: -0.0374, probability: 0.9633
  Token 11: ',' - logprob: -0.0007, probability: 0.9993
  Token 12: ' optim' - logprob: -0.0025, probability: 0.9975
  Token 13: 'izer' - logprob: -0.0020, probability: 0.9980
  Token 14: '):' - logprob: -0.6333, probability: 0.5308
  Token 15: '
' - logprob: -0.0249, prob

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


== Prompt: The capital of France is ==
Completion:  Paris.


The capital of Germany is Berlin.

The
Perplexity: 2.0134
Average logprob: -0.6998

Token breakdown:
  Token 1: ' Paris' - logprob: -0.4720, probability: 0.6237
  Token 2: '.' - logprob: -0.1885, probability: 0.8282
  Token 3: '
' - logprob: -0.2594, probability: 0.7715
  Token 4: '
' - logprob: -1.6589, probability: 0.1904
  Token 5: '
' - logprob: -3.0957, probability: 0.0452
  Token 6: 'The' - logprob: -2.1069, probability: 0.1216
  Token 7: ' capital' - logprob: -0.8047, probability: 0.4472
  Token 8: ' of' - logprob: -0.1758, probability: 0.8388
  Token 9: ' Germany' - logprob: -1.3536, probability: 0.2583
  Token 10: ' is' - logprob: -0.0085, probability: 0.9916
  Token 11: ' Berlin' - logprob: -0.0304, probability: 0.9700
  Token 12: '.' - logprob: -0.0149, probability: 0.9853
  Token 13: '
' - logprob: -0.0396, probability: 0.9612
  Token 14: '
' - logprob: -0.0408, probability: 0.9600
  Token 15: 'The' - logprob: -0